In [48]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [3]:
contracts = pd.DataFrame.from_csv("~/contracts.csv")

In [9]:
contracts.ix[724]

contractname      F.US.EPH11
month                      H
monthint                   3
year                    2011
idinstrument              11
expirationdate    2011-03-18
cqgsymbol            F.EPH11
Name: 724, dtype: object

In [10]:
options = pd.DataFrame.from_csv("~/options.csv")

In [12]:
options.ix[195035]

optionname        C.US.EPF1112650
optionmonth                     F
optionmonthint                  1
optionyear                   2011
strikeprice                  1265
callorput                       C
idinstrument                   11
expirationdate         2011-01-21
idcontract                    724
cqgsymbol          C.US.EPF111265
Name: 195035, dtype: object

In [15]:
def convert_mat_date(matlab_datenum):
    """
    Matlab to Python date convert helper function
    """
    return datetime.fromordinal(int(matlab_datenum)) + timedelta(days=int(matlab_datenum%1)) - timedelta(days=366)

In [18]:
from scipy.io import loadmat, savemat
mat = loadmat('/home/ubertrader/Downloads/strategy_210301.mat')

In [21]:
mat_date = mat['optStr']['seriesDates'][0][0][0]
dates = list(map(convert_mat_date, mat_date))

In [26]:
is_rollover = mat['optStr']['rolloverDay'][0][0][0]

In [29]:
is_rollover

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

In [89]:
def get_leg_array_ids(mat, leg_index):
    res = np.zeros_like(mat['optStr']['optLegName'][0][0][0])
    
    for i in range(len(res)):
        res[i] = mat['optStr']['optLegName'][0][0][leg_index][i][0][0]
    
    return res    

def get_leg_array_price(mat, leg_index):
    res = mat['optStr']['realLegSeries'][0][0][leg_index]
    return res    

In [90]:
leg_ids = get_leg_array_ids(mat, 0)
leg_price = get_leg_array_price(mat, 0)

In [88]:
mat['optStr']['realLegSeries'][0][0][1]

array([ 1327.25,  1327.25,  1327.25, ...,  2162.25,  2161.75,  2160.5 ])

In [91]:
def leg_contract_series(leg_ids, leg_price, dates, contracts_index):
    if len(leg_ids) != len(dates):
        raise Exception('if len(leg_ids) != len(dates):')
    contracts = []
    
    for i in range(len(leg_ids)):
        con_id = leg_ids[i]
        
        name = contracts_index.ix[con_id]['optionname']
        contracts.append(name)
        
    return pd.DataFrame({'name': contracts, 'price': leg_price}, index=dates)

In [93]:
leg_series = leg_contract_series(leg_ids, leg_price, dates, options)

In [80]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')


In [94]:
print_full(leg_series.ix['2014-01':'2014-06'])

                       name  price
2014-01-01  C.US.EPF1418000  42.00
2014-01-02  C.US.EPF1418000  36.25
2014-01-03  C.US.EPF1418000  34.75
2014-01-06  C.US.EPF1418000  29.00
2014-01-07  C.US.EPF1418000  36.75
2014-01-08  C.US.EPF1418000  35.00
2014-01-09  C.US.EPF1418000  36.25
2014-01-10  C.US.EPF1418000  37.00
2014-01-13  C.US.EPF1418000  18.25
2014-01-14  C.US.EPF1418000  33.75
2014-01-15  C.US.EPG1418400  43.75
2014-01-16  C.US.EPG1418400  23.50
2014-01-17  C.US.EPG1418400  19.75
2014-01-20  C.US.EPG1418400  20.25
2014-01-21  C.US.EPG1418400  21.00
2014-01-22  C.US.EPG1418400  22.00
2014-01-23  C.US.EPG1418400  13.75
2014-01-24  C.US.EPG1418400   6.25
2014-01-27  C.US.EPG1418400   4.60
2014-01-28  C.US.EPG1418400   4.25
2014-01-29  C.US.EPG1418400   2.40
2014-01-30  C.US.EPG1418400   5.00
2014-01-31  C.US.EPG1418400   3.65
2014-02-03  C.US.EPG1418400   0.80
2014-02-04  C.US.EPG1418400   0.80
2014-02-05  C.US.EPG1418400   0.85
2014-02-06  C.US.EPG1418400   1.10
2014-02-07  C.US.EPG